In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화
chrome_options.add_experimental_option('detach', True)  # 브라우저가 자동으로 닫히지 않도록 설정

# 키워드 목록과 날짜 범위 설정
keywords = [
    "검색어",
]
start_date_str = "2024.06.01"
end_date_str = "2024.08.31"

seen_urls = set() 

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def parse_date(date_str):
    """날짜 문자열에서 요일 제거 후 datetime 객체로 변환."""
    try:
        # 날짜 문자열에서 "(요일)" 제거
        cleaned_date_str = date_str.split('(')[0].strip()
        # "YYYY.MM.DD" 형식으로 파싱
        return datetime.strptime(cleaned_date_str, "%Y.%m.%d")
    except ValueError:
        print(f"Invalid date format: {date_str}")
        return None

def fetch_search_results(keyword, start_date, end_date):
    """키워드와 날짜 범위에 따라 기사를 스크래핑합니다."""
    base_url = "https://www.segye.com/search"
    driver.get(base_url)

    results = []
    page = 1

    # 검색어 입력
    search_box = driver.find_element(By.ID, 'searchWord')
    search_box.send_keys(keyword)

    # 검색 버튼 클릭
    search_button = driver.find_element(By.CLASS_NAME, 'imgBtn')
    search_button.click()

    # 더보기 버튼 클릭
    more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@href=\"javascript:search.goMore('article');\"]"))
    )
    driver.execute_script("arguments[0].click();", more_button)

    while True:
        # 페이지 소스 가져오기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul#articleArea")))
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기사 목록 가져오기
        articles = soup.find_all("ul", id="articleArea")

        # 기사 정보 추출
        for section in articles:
            items = section.find_all("li")
            for item in items:
                try:
                    title_tag = item.find("strong", class_="tit")
                    link_tag = item.find("a")
                    date_tag = item.find("small", class_="date")

                    if title_tag and link_tag and date_tag:
                        title = title_tag.text.strip()
                        url = link_tag["href"]
                        date_str = date_tag.text.strip()

                        # 중복 URL 체크
                        if url in seen_urls:
                            continue  # 중복된 URL은 무시
                        seen_urls.add(url)

                        # 날짜 파싱
                        article_date = parse_date(date_str)
                        if article_date is None:
                            continue  # 잘못된 날짜 형식은 스킵

                        # 시작 날짜 이전 기사면 종료
                        if article_date < start_date:
                            print(f"기사가 시작 날짜 이전입니다. 스크래핑을 중단합니다: {date_str}")
                            return results  # 루프 종료

                        # 종료 날짜 이후의 기사는 무시
                        if article_date > end_date:
                            continue  # 범위 밖의 기사는 스킵

                        # 날짜 범위 내 기사만 추가
                        if start_date <= article_date <= end_date:
                            results.append({
                                "media": '세계일보',
                                "title": title,
                                "date": date_str,
                                "section": '',
                                "article_type": '',
                                'writer': '',
                                "page": '',
                                "url": url,
                                'content': ''
                            })

                except Exception as e:
                    print(f"Error processing article: {e}")

        print(f"{keyword} - {page} 페이지를 처리")

        # 페이지 전환
        if page % 10 == 0:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.next"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break  # 더 이상 다음 페이지가 없으면 종료
        else:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//a[text()='{page + 1}']"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break  # 더 이상 다음 페이지가 없으면 종료

        page += 1

    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

# 날짜 문자열을 datetime 객체로 변환
start_date = datetime.strptime(start_date_str, "%Y.%m.%d")
end_date = datetime.strptime(end_date_str, "%Y.%m.%d")

# 모든 키워드에 대해 검색 수행
all_results = []
for keyword in keywords:
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)

# 결과를 엑셀 파일로 저장
save_to_excel(all_results, f'1세계일보 {start_date_str}~{end_date_str}.xlsx')

# 드라이버 종료
driver.quit()




In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 서브 타이틀을 스크랩하는 함수 정의
def get_content_and_subtitle_from_url(url):
    try:
        driver.get(url)

        # 페이지가 완전히 로드될 때까지 대기 (최대 10초 대기)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'contMain2'))
        )

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 서브 타이틀 추출 (em.precis)
        subtitle_tag = soup.find('em', class_='precis')
        subtitle = ""
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True)
            subtitle_tag.decompose()  # 서브 타이틀을 본문에서 제거

        # 본문 추출
        article_body = soup.find('section', id='contMain2')
        if article_body:
            # 'figure.image' 클래스를 가진 모든 태그를 제거합니다.
            for caption in article_body.find_all('figure', class_='image'):
                caption.decompose()

            content = article_body.get_text(separator='\n', strip=True)

            cut_off_text = "[ⓒ 세계일보 & Segye.com, 무단전재 및 재배포 금지]"
            if cut_off_text in content:
                content = content.split(cut_off_text)[0].strip()

        else:
            content = ""

        return content, subtitle

    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return "", ""  # 오류 발생 시 빈 문자열 반환
    
# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트
error_log = []
total_articles = len(df)

# 각 기사에 대해 본문과 서브 타이틀 스크래핑 수행
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{len(df)}: {row['url']}")
    content, subtitle = get_content_and_subtitle_from_url(row['url'])
    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 컬럼 추가

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 불필요한 항목을 제외한 후 엑셀 파일로 저장
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")
